In [ ]:
import gpxpy
import gpxpy.gpx
import pandas as pd
from pathlib import Path
from typing import List, Tuple
from haversine import haversine, Unit
import folium
from folium.plugins import MarkerCluster


# Step 1: 刪除時間 + 匯出 CSV
def remove_time_and_export_csv(gpx_file: str) -> Tuple[pd.DataFrame, gpxpy.gpx.GPX]:
    with open(gpx_file, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)

    data = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                data.append(
                    {
                        "latitude": point.latitude,
                        "longitude": point.longitude,
                        "elevation": point.elevation,
                    }
                )
                point.time = None  # 去除時間

    df = pd.DataFrame(data)
    csv_name = Path(gpx_file).stem + ".csv"
    df.to_csv(csv_name, index=False)
    print(f"CSV 匯出完成：{csv_name}")
    return df, gpx


# Step 2: 讀取通訊點
def load_communication_points(txt_file: str) -> List[Tuple[str, float, float, float]]:
    df = pd.read_csv(txt_file, sep="\t")
    points = []
    for _, row in df.iterrows():
        lat = float(str(row["緯度"]).replace("°", "").strip())
        lon = float(str(row["經度"]).replace("°", "").strip())
        ele = float(str(row["海拔（約）"]).replace("m", "").strip())
        label = f"{row['步道名稱']} {row['路標指示']}"
        points.append((label, lat, lon, ele))
    return points


# Step 3: 插入通訊點
def insert_points(
    gpx: gpxpy.gpx.GPX, comm_pts: List[Tuple[str, float, float, float]]
) -> gpxpy.gpx.GPX:
    for label, lat, lon, ele in comm_pts:
        closest_dist = float("inf")
        insert_idx = None
        segment = gpx.tracks[0].segments[0]
        points = segment.points

        for i in range(len(points) - 1):
            d1 = haversine(
                (points[i].latitude, points[i].longitude), (lat, lon), unit=Unit.METERS
            )
            d2 = haversine(
                (points[i + 1].latitude, points[i + 1].longitude),
                (lat, lon),
                unit=Unit.METERS,
            )
            if d1 + d2 < closest_dist:
                closest_dist = d1 + d2
                insert_idx = i + 1

        new_point = gpxpy.gpx.GPXTrackPoint(latitude=lat, longitude=lon, elevation=ele)
        segment.points.insert(insert_idx, new_point)
        print(f"已插入通訊點：{label}")
    return gpx


# Step 4: 切割單一路線（由第一至最後通訊點之間）
def split_single_segment(
    gpx: gpxpy.gpx.GPX,
    start_latlon: Tuple[float, float],
    end_latlon: Tuple[float, float],
) -> gpxpy.gpx.GPX:
    points = gpx.tracks[0].segments[0].points

    def find_idx(target: Tuple[float, float]) -> int:
        return min(
            range(len(points)),
            key=lambda i: haversine(
                (points[i].latitude, points[i].longitude), target, unit=Unit.METERS
            ),
        )

    idx_start = find_idx(start_latlon)
    idx_end = find_idx(end_latlon)
    if idx_start > idx_end:
        idx_start, idx_end = idx_end, idx_start

    new_gpx = gpxpy.gpx.GPX()
    new_track = gpxpy.gpx.GPXTrack()
    new_segment = gpxpy.gpx.GPXTrackSegment()
    new_segment.points = points[idx_start : idx_end + 1]
    new_track.segments.append(new_segment)
    new_gpx.tracks.append(new_track)

    print(f"切割完成：索引 {idx_start} 至 {idx_end}")
    return new_gpx


# import folium
from folium.plugins import MarkerCluster


def visualize_with_original_and_cut(
    original_gpx: gpxpy.gpx.GPX,
    sliced_gpx: gpxpy.gpx.GPX,
    comm_pts: List[Tuple[str, float, float, float]],
    output_html: str,
):
    # 初始地圖中心
    first_point = original_gpx.tracks[0].segments[0].points[0]
    m = folium.Map(
        location=[first_point.latitude, first_point.longitude], zoom_start=14
    )

    # 圖層群組
    original_layer = folium.FeatureGroup(name="原始路線", show=True)
    cut_layer = folium.FeatureGroup(name="切割後路線", show=True)
    comm_layer = folium.FeatureGroup(name="通訊點", show=True)

    # 原始路線 PolyLine
    original_coords = [
        (pt.latitude, pt.longitude) for pt in original_gpx.tracks[0].segments[0].points
    ]
    folium.PolyLine(
        original_coords, color="blue", weight=4, opacity=0.6, tooltip="原始路線"
    ).add_to(original_layer)

    # 切割後路線 PolyLine
    cut_coords = [
        (pt.latitude, pt.longitude) for pt in sliced_gpx.tracks[0].segments[0].points
    ]
    folium.PolyLine(
        cut_coords, color="red", weight=4, opacity=0.8, tooltip="切割後路線"
    ).add_to(cut_layer)

    # 通訊點 Marker
    cluster = MarkerCluster(name="通訊點群組").add_to(comm_layer)
    for label, lat, lon, ele in comm_pts:
        folium.Marker(
            [lat, lon],
            popup=f"{label}<br>海拔：約{ele} m",
            icon=folium.Icon(color="purple", icon="info-sign"),
        ).add_to(cluster)

    # 加入各層至地圖
    original_layer.add_to(m)
    cut_layer.add_to(m)
    comm_layer.add_to(m)

    # 圖層控制按鈕
    folium.LayerControl().add_to(m)

    # 儲存HTML
    m.save(output_html)
    print(f"✅ 地圖已儲存：{output_html}")


# --- 主程式 ---
if __name__ == "__main__":
    gpx_path = "標準桃山.gpx"
    txt_path = "communication_point.txt"

    # 1. 時間刪除與匯出 CSV
    df, gpx_obj = remove_time_and_export_csv(gpx_path)

    # 2. 載入與插入通訊點
    comm_pts = load_communication_points(txt_path)
    gpx_updated = insert_points(gpx_obj, comm_pts)

    # 3. 切出單一段路線
    start_latlon = (comm_pts[0][1], comm_pts[0][2])
    end_latlon = (comm_pts[-1][1], comm_pts[-1][2])
    sliced_gpx = split_single_segment(gpx_updated, start_latlon, end_latlon)

    # 4. 儲存 GPX
    with open("single_segment.gpx", "w", encoding="utf-8") as f:
        f.write(sliced_gpx.to_xml())
    print("儲存單一路線GPX：single_segment.gpx")

    # 5. 顯示完整地圖（含圖層控制）
    visualize_with_original_and_cut(
        gpx_updated, sliced_gpx, comm_pts, "route_map_with_control.html"
    )

CSV 匯出完成：標準桃山.csv
已插入通訊點：桃山瀑布步道 0 k 起點
已插入通訊點：桃山步道 0K_瀑布1K交會
已插入通訊點：桃山步道 2 K
已插入通訊點：桃山步道 4.5 K
切割完成：索引 7 至 165
儲存單一路線GPX：single_segment.gpx
✅ 地圖已儲存：route_map_with_control.html
